In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df_teams = pd.read_csv('../input/ncaaw-march-mania-2021-spread/WTeams.csv')
df_teams.head(2)

In [ ]:
df_team_conf = pd.read_csv('../input/ncaaw-march-mania-2021-spread/WTeamConferences.csv')
df_team_conf.head(2)

In [ ]:
df=pd.merge(df_teams,df_team_conf,how='outer',on='TeamID')

In [ ]:
df.isnull().sum()

In [ ]:
df.head(2)

In [ ]:
dfrs= pd.read_csv('../input/ncaaw-march-mania-2021-spread/WRegularSeasonCompactResults.csv')
dfrs.head(2)

In [ ]:
dfrs=dfrs.drop(columns=['DayNum'],axis=1)

In [ ]:
dfrs['scorediff']=dfrs['WScore']-dfrs['LScore']

In [ ]:
dfrs.head()

In [ ]:
df1=dfrs.drop(columns=['WScore','LScore'],axis=1)

In [ ]:
df1.columns

In [ ]:
df_tour = pd.read_csv('../input/ncaaw-march-mania-2021-spread/WNCAATourneyCompactResults.csv')
df_tour.head(2)

In [ ]:
df_tour=df_tour.drop(columns=['DayNum'],axis=1)

In [ ]:
df_tour['scorediff']=df_tour['WScore']-dfrs['LScore']

In [ ]:
df_tour.head()

In [ ]:
df2=df_tour.drop(columns=['WScore','LScore'],axis=1)

In [ ]:
df2.columns

In [ ]:
data=pd.merge(df1,df2,how='outer')
data.shape

In [ ]:
data.head(2)

In [ ]:
df_tour_seeds = pd.read_csv('../input/ncaaw-march-mania-2021-spread/WNCAATourneySeeds.csv')
df_tour_seeds.head(2)

In [ ]:
df_tour_seeds.shape

In [ ]:
dfwin=pd.merge(data,df_tour_seeds,left_on=['WTeamID','Season'],right_on=['TeamID','Season'])
dfwin.rename(columns={'Seed':'WSeed'}, inplace=True)
dfwin.head(2)

In [ ]:
wnew=dfwin.drop(columns=['TeamID'],axis=1)

In [ ]:
wnew.shape

In [ ]:
dfloose=pd.merge(data,df_tour_seeds,left_on=['LTeamID','Season'],right_on=['TeamID','Season'])
dfloose.rename(columns={'Seed':'LSeed'}, inplace=True)
dfloose.head(2)
#'WLoc':'LLoc','scorediff':'LScore','NumOT':'LNumOT'

In [ ]:
dfloose.shape

In [ ]:
lnew=dfloose.drop(columns=['TeamID','WLoc','scorediff','NumOT'],axis=1)

In [ ]:
df_new=pd.merge(wnew,lnew,on=['WTeamID','LTeamID','Season'],how='inner')
df_new.head(2)

In [ ]:
df_new.rename(columns={'WTeamID':'TeamIdA','LTeamID':'TeamIdB'},inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
df_new['WSeed'] = label.fit_transform(df_new['WSeed'])# categories classified
df_new['LSeed'] = label.fit_transform(df_new['LSeed'])# categories classified
df_new['WLoc'] = label.fit_transform(df_new['WLoc'])# categories classified

In [ ]:
df_new['DSeed']=df_new['WSeed']-df_new['LSeed']

In [ ]:
df_new.columns

In [ ]:
df_new.shape

In [ ]:
features=['TeamIdA','TeamIdB','WSeed','LSeed','Season']

In [ ]:
X=df_new[features]

In [ ]:
y=df_new['scorediff']

In [ ]:
#### Split data on train and test
import sklearn.model_selection as model_selection
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,test_size=0.20,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf=RandomForestRegressor() 
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
param_dict = {'n_estimators': (100,200,300), 'min_samples_split': np.arange(3, 10), 
              'min_samples_leaf': np.arange(1,5),'max_features':['auto', 'sqrt', 'log2']}

In [ ]:
rng = np.random.RandomState(0)

from sklearn.model_selection import RandomizedSearchCV
rsh = RandomizedSearchCV(estimator=clf, param_distributions=param_dict, random_state=rng,cv=5)

rsh.fit(X_train, y_train)
rsh.best_params_

In [ ]:
clf1=RandomForestRegressor(n_estimators= 300,
 min_samples_split= 6,
 min_samples_leaf= 4,
 max_features= 'sqrt') 
clf1.fit(X_train, y_train)

In [ ]:
clf1.score(X_test,y_test)

In [ ]:
df_test = pd.read_csv( '../input/ncaaw-march-mania-2021-spread/WSampleSubmissionStage1.csv')
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIdA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))

In [ ]:
df_test.shape

In [ ]:
datatest=df_test.drop(columns=['ID','Pred'],axis=1)
datatest.head()

In [ ]:
dftestwin=pd.merge(datatest,df_tour_seeds,left_on=['TeamIdA','Season'],right_on=['TeamID','Season'])
dftestwin.rename(columns={'Seed':'WSeed'}, inplace=True)
dftestwin.head(2)

In [ ]:
dftestwin.shape

In [ ]:
data1=dftestwin.drop(columns=['TeamID'],axis=1)

In [ ]:
dftestloose=pd.merge(datatest,df_tour_seeds,left_on=['TeamIdB','Season'],right_on=['TeamID','Season'])
dftestloose.rename(columns={'Seed':'LSeed'}, inplace=True)
dftestloose.head(2)

In [ ]:
dftestloose.shape

In [ ]:
data2=dftestloose.drop(columns=['TeamID'],axis=1)

In [ ]:
dftest=pd.merge(data1,data2,left_on=['TeamIdA','TeamIdB','Season'],right_on=['TeamIdA','TeamIdB','Season'])
dftest.head(2)

In [ ]:
dftest.shape

In [ ]:
dftest.isnull().sum()

In [ ]:
dftest['WSeed'] = label.fit_transform(dftest['WSeed'])# categories classified
dftest['LSeed'] = label.fit_transform(dftest['LSeed'])# categories classified

In [ ]:
test=dftest[features]

In [ ]:
predtest=clf1.predict(test)

In [ ]:
test['preds']=predtest
test['actuals']=df_new['scorediff']

In [ ]:
import plotly.express as px
fig=px.line(test,x=test.index,y="preds")
fig=fig.add_scatter(x=test.index,y=test['actuals'],name="actual")
fig

In [ ]:
test.head()

In [ ]:
subtest=test[['preds','Season','TeamIdA','TeamIdB']]

In [ ]:
subtest['ID'] = subtest[subtest.columns[1:]].apply(
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
subtest.head()

In [ ]:
submisson=subtest[['ID','preds']]

In [ ]:
submisson.shape

In [ ]:
submisson.to_csv('Submisson.csv',index=False)